# Fluxo de Trabalho de Machine Learning

Neste notebook veremos com mais detalhes o fluxo de trabalho de Machine Learning com a biblioteca **Scikit-Learn**.

Abordaremos os seguintes tópicos:

1. Preparando os Dados
2. Escolhendo o melhor Modelo/Estimador/Algoritmo para os nossos problemas
3. Ajustar o Modelo e usá-lo para fazer previsões em nossos dados
4. Avaliar o desempenho do Modelo
5. Aperfeiçoar o Modelo
6. Salvar e carregar o Modelo treinado

Antes de tudo, vamos importar as bibliotecas necessárias para trabalharmos adequadamente.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Deixando os Dados prontos para serem usados com Machine Learning

Para esta seção devemos executar três principais etapas:

1. Dividir os dados em features (características) e labels (rótulos) (Normalmente `X` e `y`)
2. Preencher (também conhecido como imputing) ou eliminar os valores faltantes (missing)
3. Converter valores não-numéricos para valores numéricos (também conhecido como feature encoding)

Carregaremos os nossos dados experimentais sobre doença do coração (algumas pessoas possuem e outras não).

Em seguida vamos visualizar os primeiros dados da tabela com a função **head()**.

In [2]:
doença_coração = pd.read_csv('dados/heart-disease.csv')
doença_coração.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Agora vamos dividir os dados em **X** e **y**.

**X** é o nosso conjunto de features, portando vamos eliminar apenas a coluna **target**, que se trata de **y**, as labels que desejamos prever.

In [3]:
X = doença_coração.drop('target',axis=1)
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


E agora finalmente selecionamos apenas a coluna **target** (**y**).

In [4]:
y = doença_coração['target']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

Agora vamos dividir os dados em conjuntos de **treinamento** e **teste**. 

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Agora podemos verificar a forma das matrizes que obtivemos para entendermos melhor com os dados que estamos trabalhando.

In [6]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(242, 13) (61, 13) (242,) (61,)


Temos então respectivamente:

- 242 linhas e 13 colunas para **X_train** (80% para treinamento)
- 61 linhas e 13 colunas para **X_test** (20% para teste)
- 242 linhas para **y_train** (80% para treinamento)
- 61 linhas para **y_test** (20% para teste)

Podemos tirar a prova real para confirmar que está tudo certo:

In [8]:
int(X.shape[0] * 80/100)

242

In [13]:
242 + 61

303

In [14]:
len(doença_coração)

303

### Trabalhando com Dados Não-Numéricos

Agora vamos usar outro conjunto de dados para fazermos algumas verificações.

Trata-se de um conjunto de dados de carros, vamos carregá-lo com a biblioteca pandas através da função **read_csv()**.

In [15]:
carros = pd.read_csv('dados/car-sales-extended.csv')
carros.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431,4,15323
1,BMW,Blue,192714,5,19943
2,Honda,White,84714,4,28343
3,Toyota,White,154365,4,13434
4,Nissan,Blue,181577,3,14043


Quantos carros temos em nosso conjunto de dados?

In [16]:
len(carros)

1000

Com quais tipos de dados estamos trabalhando?

In [17]:
carros.dtypes

Make             object
Colour           object
Odometer (KM)     int64
Doors             int64
Price             int64
dtype: object

Observe que nem todos os dados são numéricos, e para usarmos os dados em um algoritmo de Machine Learning, precisamos que eles sejam todos numéricos.

Novamente vamos dividir os dados em **X** & **y**.

In [18]:
X = carros.drop('Price',axis=1)
y = carros['Price']

E dividir em **treinamento** e **teste**.

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

Vamos agora transformar as categorias (strings) em números para podermos alimentar o algoritmo de Machine Learning, também conhecido como Modelo ou Estimador.

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ['Make','Colour','Doors']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot',one_hot,categorical_features)],remainder='passthrough')
transformed_X = transformer.fit_transform(X)
transformed_X

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 3.54310e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.00000e+00, 1.92714e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 8.47140e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 6.66040e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.15883e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.48360e+05]])

Observe que nos é retornado um array **numpy**.

A partir dele podemos construir um DataFrame **pandas**.

In [21]:
pd.DataFrame(transformed_X).head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,42652.0
6,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,163453.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,43120.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,130538.0
9,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,51029.0


Como podemos ver, temos 4 marcas diferentes de carros, 5 cores diferentes e 3 modelos de portas. 

Por este motivo o **OneHotEncoder** nos trouxe 12 colunas e apenas 3 estão marcadas com **1**, indicando uma Marca, Cor e Modelo de Porta para cada carro específico.

In [22]:
carros['Make'].value_counts()

Toyota    398
Honda     304
Nissan    198
BMW       100
Name: Make, dtype: int64

In [23]:
carros['Colour'].value_counts()

White    407
Blue     321
Black     99
Red       94
Green     79
Name: Colour, dtype: int64

In [24]:
carros['Doors'].value_counts()

4    856
5     79
3     65
Name: Doors, dtype: int64

Outra forma de fazer essa codificação é com a própria biblioteca **pandas**, através do método **get_dummies()**.

In [25]:
dummies = pd.get_dummies(carros[['Make','Colour','Doors']])
dummies.head(10)

,Doors,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White
0,4,0,1,0,0,0,0,0,0,1
1,5,1,0,0,0,0,1,0,0,0
2,4,0,1,0,0,0,0,0,0,1
3,4,0,0,0,1,0,0,0,0,1
4,3,0,0,1,0,0,1,0,0,0
5,4,0,1,0,0,0,0,0,1,0
6,4,0,0,0,1,0,1,0,0,0
7,4,0,1,0,0,0,0,0,0,1
8,4,0,0,1,0,0,0,0,0,1
9,4,0,1,0,0,0,1,0,0,0


Podemos agora finalmente construir o nosso Modelo.

Observe que dessa vez estamos trabalhando com um **Regressor**, pois queremos prever dados numéricos (preço do carro).

Primeiramente dividimos os dados em **treinamento** e **teste**.

In [26]:
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.2)

E agora instanciamos e treinamos o Modelo que vamos utilizar.

In [27]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

É possível vermos que ele nos retorna os hiperparâmetros utilizados, neste caso específico, usamos todos os hiperparâmetros padrões para este modelo.

E agora avaliamos o desempenho do modelo através do método **score()**.

In [28]:
model.score(X_test,y_test)

0.2651919526682347

Perceba que obtivemos uma pontuação ruim para este modelo, isso ocorre pelo fato da limitação de features em nossos dados, insuficientes para que o algoritmo seja capaz de ter um bom desempenho e possa nos trazer boas previsões.

### Trabalhando com Dados Faltantes (Missing Data)

O que ocorre se houver dados faltantes (**missing**) em nosso conjunto de dados?

A ideia para solucionar este problema seria:

1. Preencher eles com algum valor (também conhecido como imputation)
2. Remover as amostras que estejam com dados faltantes

Para experimentar com este tipo de problema de dados faltantes, vamos carregar um conjunto de dados que esteja com dados faltando.

In [31]:
carros_missing = pd.read_csv('dados/car-sales-extended-missing-data.csv')
carros_missing.tail(15)

,Make,Colour,Odometer (KM),Doors,Price
985,NaN,Blue,216250.0,4.0,9691.0
986,Honda,White,71934.0,4.0,26882.0
987,Honda,White,215235.0,4.0,3825.0
988,Nissan,Black,248736.0,4.0,8358.0
989,Toyota,Red,41735.0,4.0,13928.0
990,Toyota,White,173408.0,4.0,8082.0
991,Honda,Blue,235985.0,4.0,9184.0
992,Honda,Green,54721.0,4.0,27419.0
993,Nissan,Black,162523.0,4.0,4696.0
994,BMW,Blue,163322.0,3.0,31666.0


Perceba que algumas células possuem a entrada **NaN**, indicando que ali há um espaço vazio.

Existe uma função da biblioteca **pandas** que nos reporta quantos dados faltantes existem em nosso conjunto de dados.

In [32]:
carros_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

Como podemos ver, temos aproximadamente 50 dados faltantes em cada coluna.

Para lidar com este problema temos algumas opções.

#### **Opção 1**: Preencher os dados faltantes com pandas.

Podemos por exemplo preencher as colunas em que os dados estão faltando.

In [33]:
carros_missing['Make'].fillna('missing',inplace=True)
carros_missing['Colour'].fillna('missing',inplace=True)
carros_missing['Odometer (KM)'].fillna(carros_missing['Odometer (KM)'].mean(),inplace=True)
carros_missing['Doors'].fillna(4,inplace=True)

As colunas **Make** e **Colour** serão preenchidas com a string 'missing'.

A coluna **Odometer (KM)** será preenchida com a média dos dados dessa coluna.

A coluna **Doors** será preenchida com o número **4**.

Novamente checamos o DataFrame para ver se está tudo certo.

In [34]:
carros_missing.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

Nos resta a coluna **Price**, para ela, removeremos as linhas com valores faltantes.

In [35]:
carros_missing.dropna(inplace=True)

E agora confirmamos que está tudo certo.

In [36]:
carros_missing.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

#### **Opção 2**: Preencher os dados faltantes com Scikit-Learn

Vamos então recarregar os dados para voltarmos ao seu estado original com dados faltantes.

In [39]:
carros_missing = pd.read_csv('dados/car-sales-extended-missing-data.csv')
carros_missing.head(9)

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
5,Honda,Red,42652.0,4.0,23883.0
6,Toyota,Blue,163453.0,4.0,8473.0
7,Honda,White,NaN,4.0,20306.0
8,NaN,White,130538.0,4.0,9374.0


Confirmando que temos dados faltantes em nosso conjunto de dados.

In [40]:
carros_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

Removeremos os dados faltantes da coluna **Price**.

In [41]:
carros_missing.dropna(subset=['Price'],inplace=True)

E novamente verificamos quantos dados faltantes nos resta.

In [42]:
carros_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

Agora iremos dividir os dados em **X** e **y**.

In [43]:
X = carros_missing.drop('Price',axis=1)
y = carros_missing['Price']

Podemos então preencher os dados faltantes com a biblioteca **Scikit-Learn**.

Preencheremos os valores categóricos com a palavra **'missing'** e os valores numéricos com a **média**.

In [44]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

categorical_imputer = SimpleImputer(strategy='constant',fill_value='missing')
door_imputer = SimpleImputer(strategy='constant',fill_value=4)
numerical_imputer = SimpleImputer(strategy='mean')

Definimos as colunas.

In [45]:
categorical_features = ['Make','Colour']
door_features = ['Doors']
numerical_features = ['Odometer (KM)']

Criamos um **Imputer** (que preencherá os dados faltantes).

In [46]:
imputer = ColumnTransformer([
    ('categorical_imputer',categorical_imputer,categorical_features),
    ('door_imputer',door_imputer,door_features),
    ('numerical_imputer',numerical_imputer,numerical_features)
])

E agora podemos transformar os dados.

In [47]:
filled_X = imputer.fit_transform(X)
filled_X

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)

Nos foi retornado um array **numpy**, vamos colocar estes dados em um DataFrame.

In [49]:
carros_missing_filled = pd.DataFrame(filled_X, columns=['Make','Colour','Doors','Odometer (KM)'])
carros_missing_filled.head(9)

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4,35431
1,BMW,Blue,5,192714
2,Honda,White,4,84714
3,Toyota,White,4,154365
4,Nissan,Blue,3,181577
5,Honda,Red,4,42652
6,Toyota,Blue,4,163453
7,Honda,White,4,130987
8,missing,White,4,130538


Confirmamos se obtivemos sucesso em preencher os dados faltantes.

In [50]:
carros_missing_filled.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

Lembre que devemos transformar os dados categóricos em numéricos com o **OneHotEncoder**.

In [51]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ['Make','Colour','Doors']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot',one_hot,categorical_features)],remainder='passthrough')
transformed_X = transformer.fit_transform(carros_missing_filled)
transformed_X

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

Agora temos os nossos dados como números e preenchidos (sem dados faltantes, também conhecidos como missing data).

Vamos fazer o split de **treinamento** e **teste** e ajustar o modelo.

In [59]:
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.2)

model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.14236308128488007

Novamente o nosso modelo está performando ruim, pior até mesmo que anterior que testamos, e o motivo se deve ao fato de que estamos usando uma quantidade menor de dados, e isso, normalmente em Machine Learning leva o algoritmo a ter uma perfomance limitada.

In [61]:
print(len(carros_missing_filled), len(carros))

950 1000


Em resumo: o processo de preencher valores faltantes se chama **Imputation** e o processo de transformar dados não-numéricos em numéricos é chamado de **Feature Engineering** ou **Feature Encoding**.

Agora podemos ir para a pŕoxima etapa de nosso Workflow.